In [17]:
import numpy as np
import pandas as pd
import os
from IPython.display import display
import tensorflow as tf
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.layers import Layer
import re

In [56]:
# Data Load

from retrieve_data import *


cornell_utt, cornell_res = cornell('data/cornell movie-dialogs corpus')
himym_utt, himym_res = himym('data/HIMYM')

print(len(cornell_utt))
print(len(cornell_res))

print(len(himym_utt))
print(len(himym_utt))

30000
30000
3797
3797


In [23]:
'''
ans = np.append(cornell_res, himym_res)
ques = np.append(cornell_utt, himym_utt)
'''

In [57]:
###  count occurences ###

ques = cornell_utt
ans = cornell_res

word2count = {}

for line in ques:
    for word in line.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1
for line in ans:
    for word in line.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1



###  remove less frequent ###
thresh = 5

vocab = {}
word_num = 0
for word, count in word2count.items():
    if count >= thresh:
        vocab[word] = word_num
        word_num += 1
        


for i in range(len(ans)):
    ans[i] = '<SOS> ' + ans[i] + ' <EOS>'


tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']

x = len(vocab)

for token in tokens:
    vocab[token] = x
    x += 1
    

vocab['cameron'] = vocab['<PAD>']
vocab['<PAD>'] = 0


### inv answers dict ###
inv_vocab = {w:v for v, w in vocab.items()}


encoder_inp = []
for line in ques:
    lst = []
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])
        
    encoder_inp.append(lst)

decoder_inp = []
for line in ans:
    lst = []
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])        
    decoder_inp.append(lst)



from tensorflow.keras.preprocessing.sequence import pad_sequences

encoder_inp = pad_sequences(encoder_inp, 13, padding='post', truncating='post')
decoder_inp = pad_sequences(decoder_inp, 13, padding='post', truncating='post')


decoder_final_output = []

for i in decoder_inp:
    decoder_final_output.append(i[1:]) 

decoder_final_output = pad_sequences(decoder_final_output, 13, padding='post', truncating='post')

In [58]:
VOCAB_SIZE = len(vocab)
MAX_LEN = 13

print(decoder_final_output.shape, decoder_inp.shape, encoder_inp.shape, len(vocab), len(inv_vocab), inv_vocab[0])

(30000, 13) (30000, 13) (30000, 13) 3166 3165 <PAD>


In [59]:
from tensorflow.keras.utils import to_categorical

decoder_final_output = to_categorical(decoder_final_output, len(vocab))
decoder_final_output.shape

(30000, 13, 3166)

In [60]:
embeddings_index = {}

with open('data/glove.6B.50d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

In [61]:
embedding_dimention = 50

def embedding_matrix_creater(embedding_dimention, word_index):
    embedding_matrix = np.zeros((len(word_index)+1, embedding_dimention))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
          # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

embedding_matrix = embedding_matrix_creater(50, word_index=vocab)   

In [62]:
embedding_matrix.shape

(3167, 50)

In [63]:
embedding_matrix[0]

array([ 4.18000013e-01,  2.49679998e-01, -4.12420005e-01,  1.21699996e-01,
        3.45270008e-01, -4.44569997e-02, -4.96879995e-01, -1.78619996e-01,
       -6.60229998e-04, -6.56599998e-01,  2.78430015e-01, -1.47670001e-01,
       -5.56770027e-01,  1.46579996e-01, -9.50950012e-03,  1.16579998e-02,
        1.02040000e-01, -1.27920002e-01, -8.44299972e-01, -1.21809997e-01,
       -1.68009996e-02, -3.32789987e-01, -1.55200005e-01, -2.31309995e-01,
       -1.91809997e-01, -1.88230002e+00, -7.67459989e-01,  9.90509987e-02,
       -4.21249986e-01, -1.95260003e-01,  4.00710011e+00, -1.85939997e-01,
       -5.22870004e-01, -3.16810012e-01,  5.92130003e-04,  7.44489999e-03,
        1.77780002e-01, -1.58969998e-01,  1.20409997e-02, -5.42230010e-02,
       -2.98709989e-01, -1.57490000e-01, -3.47579986e-01, -4.56370004e-02,
       -4.42510009e-01,  1.87849998e-01,  2.78489990e-03, -1.84110001e-01,
       -1.15139998e-01, -7.85809994e-01])

In [65]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input, Bidirectional, Concatenate, Dropout, Attention


embed = Embedding(VOCAB_SIZE+1, 50, 
                  input_length=13,
                  trainable=True)

embed.build((None,))

embed.set_weights([embedding_matrix])

In [66]:
from attention import AttentionLayer

# Model
enc_inp = Input(shape=(13, ))


#embed = Embedding(VOCAB_SIZE+1, 50, mask_zero=True, input_length=13)(enc_inp)
enc_embed = embed(enc_inp)
enc_lstm = Bidirectional(LSTM(400, return_state=True, dropout=0.05, return_sequences = True))

encoder_outputs, forward_h, forward_c, backward_h, backward_c = enc_lstm(enc_embed)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

enc_states = [state_h, state_c]


dec_inp = Input(shape=(13, ))
dec_embed = embed(dec_inp)
dec_lstm = LSTM(400*2, return_state=True, return_sequences=True, dropout=0.05)
output, _, _ = dec_lstm(dec_embed, initial_state=enc_states)

# attention
attn_layer = AttentionLayer()
attn_op, attn_state = attn_layer([encoder_outputs, output])
decoder_concat_input = Concatenate(axis=-1)([output, attn_op])


dec_dense = Dense(VOCAB_SIZE, activation='softmax')
final_output = dec_dense(decoder_concat_input)

model = Model([enc_inp, dec_inp], final_output)

model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 13)]         0                                            
__________________________________________________________________________________________________
input_13 (InputLayer)           [(None, 13)]         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 13, 50)       158350      input_13[0][0]                   
                                                                 input_14[0][0]                   
__________________________________________________________________________________________________
bidirectional_4 (Bidirectional) [(None, 13, 800), (N 1443200     embedding_3[0][0]          

In [67]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [68]:
model.fit([encoder_inp, decoder_inp], decoder_final_output, epochs=10, batch_size=24, validation_split=0.15)

Epoch 1/10
1063/1063 [==============================] - 47s 39ms/step - loss: 1.2651 - acc: 0.8033 - val_loss: 0.9504 - val_acc: 0.8385
Epoch 2/10
1063/1063 [==============================] - 40s 37ms/step - loss: 0.9202 - acc: 0.8360 - val_loss: 0.9142 - val_acc: 0.8445
Epoch 3/10
1063/1063 [==============================] - 40s 37ms/step - loss: 0.8580 - acc: 0.8393 - val_loss: 0.9022 - val_acc: 0.8448
Epoch 4/10
1063/1063 [==============================] - 39s 37ms/step - loss: 0.8128 - acc: 0.8424 - val_loss: 0.8995 - val_acc: 0.8473
Epoch 5/10
1063/1063 [==============================] - 40s 37ms/step - loss: 0.7556 - acc: 0.8466 - val_loss: 0.9020 - val_acc: 0.8473
Epoch 6/10
1063/1063 [==============================] - 40s 38ms/step - loss: 0.6998 - acc: 0.8518 - val_loss: 0.9170 - val_acc: 0.8476
Epoch 7/10
1063/1063 [==============================] - 40s 37ms/step - loss: 0.6349 - acc: 0.8597 - val_loss: 0.9364 - val_acc: 0.8471
Epoch 8/10
1063/1063 [==========================

In [69]:
model.save('model_03.h5')

In [70]:
enc_model = tf.keras.models.Model(enc_inp, [encoder_outputs, enc_states])


decoder_state_input_h = tf.keras.layers.Input(shape=( 400 * 2,))
decoder_state_input_c = tf.keras.layers.Input(shape=( 400 * 2,))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]


decoder_outputs, state_h, state_c = dec_lstm(dec_embed , initial_state=decoder_states_inputs)


decoder_states = [state_h, state_c]

#decoder_output = dec_dense(decoder_outputs)

dec_model = tf.keras.models.Model([dec_inp, decoder_states_inputs],
                                      [decoder_outputs] + decoder_states)

In [71]:
from keras.preprocessing.sequence import pad_sequences

print("##########################################")
print("#     Start chatting with chatbot :)     #")
print("##########################################")


prepro1 = ""
while prepro1 != 'q':
    
    prepro1 = input("Me : ")
    try:
        prepro1 = clean_text(prepro1)
        prepro = [prepro1]
        
        txt = []
        for x in prepro:
            lst = []
            for y in x.split():
                try:
                    lst.append(vocab[y])
                except:
                    lst.append(vocab['<OUT>'])
            txt.append(lst)
        txt = pad_sequences(txt, 13, padding='post')


        enc_op, stat = enc_model.predict( txt )

        empty_target_seq = np.zeros( ( 1 , 1) )
        empty_target_seq[0, 0] = vocab['<SOS>']
        stop_condition = False
        decoded_translation = ''


        while not stop_condition :

            dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + stat )

            ###########################
            attn_op, attn_state = attn_layer([enc_op, dec_outputs])
            decoder_concat_input = Concatenate(axis=-1)([dec_outputs, attn_op])
            decoder_concat_input = dec_dense(decoder_concat_input)
            ###########################

            sampled_word_index = np.argmax( decoder_concat_input[0, -1, :] )

            sampled_word = inv_vocab[sampled_word_index] + ' '

            if sampled_word != '<EOS> ':
                decoded_translation += sampled_word           


            if sampled_word == '<EOS> ' or len(decoded_translation.split()) > 13:
                stop_condition = True

            empty_target_seq = np.zeros( ( 1 , 1 ) )  
            empty_target_seq[ 0 , 0 ] = sampled_word_index
            stat = [ h , c ] 

        print("chatbot : ", decoded_translation )
        print("=" * 60)

    except:
        print("sorry didn't got you , please type again :( ")

##########################################
#     Start chatting with chatbot :)     #
##########################################
Me : hi
chatbot :  hi 
Me : who are you?
chatbot :  i am just <OUT> 
Me : good to meet you
chatbot :  <OUT> <OUT> 
Me : can you say something to me?
chatbot :  yes 
Me : then just say it!
chatbot :  i am not 
Me : not what?
chatbot :  you know something 
Me : No I don't
chatbot :  you remember 
Me : remember what?
chatbot :  what 
Me : are you kidding me?
chatbot :  yes 
Me : okay that was funny
chatbot :  run into josh 
Me : who is josh
chatbot :  what 
Me : okay bye
chatbot :  okay 


KeyboardInterrupt: ignored